Importing necessary libraries for work

In [ ]:
import pandas as pd
import numpy as np
import zipfile
import sqlite3
from datetime import datetime


# Unzipping the file
Reading the zipfile with `zipfile` and its module `ZipFile`, Extracting everything to `content` folder

In [ ]:
with zipfile.ZipFile("/content/online_retail.csv.zip") as zipf:
  zipf.extractall("/content")


# CSV Reading
Reading the csv using `pandas` module `read_csv`, displaying the first 5 rows of the read dataset with `df.head()`, clearing the `CustomerID` subset using `dropna`. Converting dates to datetime using `pd.to_datetime`

In [ ]:
df=pd.read_csv("/content/online_retail.csv")
print(df.head())
df=df.dropna(subset="CustomerID")
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['InvoiceDate'] = df['InvoiceDate'].dt.strftime("%Y-%m-%d")
print(df.head())

  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

           InvoiceDate  UnitPrice  CustomerID         Country  
0  2010-12-01 08:26:00       2.55     17850.0  United Kingdom  
1  2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
2  2010-12-01 08:26:00       2.75     17850.0  United Kingdom  
3  2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
4  2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  

# Connecting to SQL
Connecting to an temporary sqlite3 server using early-imported module sqlite3. Converting the dataframe to a SQL query so I can work with it using SQL

In [ ]:
conn=sqlite3.connect(":memory:")
df.to_sql("Table", conn, index=True,if_exists="replace")

406829

# Working with the query
Selecting CustomerID, calculating Recency using `JULIANDAY`, Frequency with no duplicates and Monetary multiplying Quantity and Price

In [ ]:
query="""
SELECT
    CustomerID,
    JULIANDAY((SELECT MAX(InvoiceDate) FROM "Table")) - JULIANDAY(MAX(InvoiceDate)) AS Recency,
    COUNT(DISTINCT InvoiceNo) AS Frequency,
    SUM(Quantity*UnitPrice) AS Monetary
FROM "Table"
WHERE CustomerID IS NOT NULL
GROUP BY CustomerID;
"""
data=pd.read_sql_query(query,conn)
print(data.head())

   CustomerID  Recency  Frequency  Monetary
0     12346.0    325.0          2      0.00
1     12347.0      2.0          7   4310.00
2     12348.0     75.0          4   1797.24
3     12349.0     18.0          1   1757.55
4     12350.0    310.0          1    334.40


# Plotly work
Creating 3 Histograms with plotly: Recency, Frequency, and Monetary.

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

hist_fig = px.histogram(
    data["Recency"],
    x="Recency",
    nbins=30,
    title="Recency Distribution",
    labels={"Recency": "Buying Recency"}
)
hist_fig1 = px.histogram(
    data["Frequency"],
    x="Frequency",
    nbins=30,
    title="Frequency Distribution",
    labels={"Frequency": "Buying Frequency"}
)
hist_fig2 = px.histogram(
    data["Monetary"],
    x="Monetary",
    nbins=30,
    title="Monetary Distribution",
    labels={"Monetary": "Buying Monetary"}
)
hist_fig.show()
hist_fig1.show()
hist_fig2.show()